# Shallow water equations

Some notes on the shallow water model used in SWIM

## Mathematical formulation

The equations form a hyperbolic conservation law for water depth $h$ and horizontal momentum $hu$ and $hv$, given depth-averaged velocities $u$ and $v$ in $x$- and $y$-direction, respectively.
The equations are then given by 
$$\begin{bmatrix} h \\ hu \\ hv \end{bmatrix}_t + 
\begin{bmatrix} hu \\ hu^2 + \tfrac{1}{2}gh^2 \\ huv \end{bmatrix}_x + 
\begin{bmatrix} hv  \\ huv \\ hv^2 + \tfrac{1}{2}gh^2  \end{bmatrix}_y =
\begin{bmatrix} 0 \\ -ghB_x \\ -ghB_y \end{bmatrix}_x + \bm{S}(h, u, v). $$
where $g$ is the gravitational constant and $B$ is the height of the topography above a reference level. 

Further source terms, such as friction, rain, infiltration, etc, are for now collected in $\bm{S}(h, u, v)$ as
$$\bm{S}(h, u, v) = \bm{S}_f(h, u, v) + \bm{S}_r(h, u, v) + \bm{S}_i(h, u, v)$$

To obtain a well-balanced finite volume scheme, we construct our numerical method in terms of the altitude of the water surface $w(x,y) = h(x,y) + B(x,y)$ rather than the conserved water amount of water $h$. This leads to the following vectorized equations 
$$\bm{q}_t + \bm{F}(\bm{q})_x + \bm{G}(\bm{q})_y = \bm{S}_B(\bm{q}) + \bm{S}(\bm{q})$$
with $\bm{q} = [w, hu, hv]$.




### Source terms
In the following, we look at source terms for friction, rain, and infiltration.

#### Friction
Different formulations for the friction exists. Here are a few:
* Brodtkorb, et al. (2012)  uses 
$$ \bm{S}_f = \left[0, \quad -\frac{gn^2u}{h^{1/3}}\sqrt{u^2 + v^2}, \quad -\frac{gn^2v}{h^{1/3}}\sqrt{u^2 + v^2}\right]^T$$
with Manning roughtness coefficent $ n = 0.033\;\text{m}^{1/3}\text{s}$

* In the GPU Ocean description by Brodtkorb, Holm (2021), we used
$$ \bm{S}_f = \left[0, \quad -\frac{ru}{h}\sqrt{u^2 + v^2}, \quad -\frac{rv}{h}\sqrt{u^2 + v^2}\right]^T$$
without ever specifying the parameter $r$...

* In Fernandez-Pato, et al (2016) they use
$$ \bm{S}_f = \left[0, \quad -\frac{n^2 u}{h^{4/3}}\sqrt{u^2 + v^2}, \quad -\frac{n^2 v}{h^{4/3}}\sqrt{u^2 + v^2}\right]^T$$
with Manning roughness coefficient $n = 0.03\;\text{s/m}^{1/3}$

* In Brodtkorb et al (2010) they use a linear friction term
$$\bm{S}_f = \left[0, \quad -u\frac{\alpha h}{1 + \beta h}, \quad -v\frac{\alpha h}{1 + \beta h}\right]^T$$
with $\alpha = 10^{-2}$ and $\beta = 10^2$ chosen "rather haphazardly".


#### Rain
Rain is fairly simple, as it is simply an addition of water based on the rain rate $r$ given in m/s, meaning that 
$$ \bm{S}_r(h, u, v) = [r, 0, 0]^T.$$
Note that the rain will typically be time dependent, meaning $r(t)$.

#### Infiltration
Similar to rain, infiltration is quite simple as it can be considered as removal of water from the system. We consider an infiltration rate $f$ and a source term 
$$ \bm{S}_f(h, u, v) = [f, 0, 0]^T.$$
The infiltration rate $f$ can be modeled in a wide range of ways, and might depend on location, time, and/or the conserved variables.

Fernandez-Pato et al (2016) compare two different infiltration models. The first one, **the Horton model** is purely empirical and consider a purely time-dependent rate
$$f(t) = f_c + (f_0 - f_c)e^{-kt},$$
in which $f_0$ and $f_c$ are the initial and finla infiltration capacities, respectively, $k$ is some rate decay parameter that controls how slow or fast the infiltration rate changes.
One weakness of this model could be that if a simulation begins with heavy or no rain, the infiltration changes with time in the same way. 

The other model is the **Green-Ampt model**, which gives more sense physically, saying$
$$ f(t) = K_s + \frac{K_s \Psi \Delta \theta}{F(t)}.$$
Here, $K_s$ is the saturated hydraulic conductivity, $\Psi$ some average suction head, $\Delta \theta$ is the difference between the soil porosity and the initial volumetric water content, and $F(t) = \int_0^t f(t) dt$ is the accumulated infiltration.
This nonlinear relation is more physical, but also more complicated to solve as it requires us to keep track of the accumulated infiltrated water at each location and requires us to do a type of Newton iteration to obtain $f(t)$.





## Numerical method

To this end, we use the high-resolution finite-volume scheme given by Kurganov and Petrova from 2007 on a Cartesian grid, with $\bm{Q}_{i,j}$ defined as the cell-average values of $\bm{q}$ in the cell with index $(i, j)$.
The bottom topography is represented as a piecewise bilinear function defined through values at the cell intersections, such as $B_{i+1/2, j+1/2}$. 

The flux terms are computed by a well-balanced, positivity preserving, central-upwind method, so that the numerical fluxes balance the bathymetry source terms to preserve lake-at-rest steady state solutions, even with discontinous bottom topography.


### Semi-implicit formulation
To account for the friction, we use a semi-implicit formulation, as also used by Brodtkorb et al (2012).
We have the semi-discrete equation
$$\frac{d\bm{Q}_{i,j}}{dt} = \bm{R}(\bm{Q})_{i,j} + \bm{S}_f(\bm{Q}_{i,j}),  $$
where $\bm{R}$ contains the fluxes and all source terms except for the friction. Note that we can write the friction term as
$$\bm{S}_f(\bm{Q}_{i,j}) = \bm{Q}_{i,j} \tilde{\bm{S}_f}(\bm{Q}_{i,j}),$$
so that, $\tilde{\bm{S}_f}(\bm{Q}) = [0, s_f, s_f]^T$, where $s_f$ is, for each formulation above,
* Brodtkorb et al (2012): $s_f = -\frac{gn^2}{h^{4/3}}\sqrt{u^2 + v^2}$,
* Brodtkorb, Holm (2021): $s_f = -\frac{ru}{h^2}\sqrt{u^2 + v^2}$, and
* Fernandez-Pato, et al (2016): $s_f = -\frac{n^2 u}{h^{7/3}}\sqrt{u^2 + v^2}$

We then evaluate the friction term (in the case of a first-order temporal method) as
$$\bm{S}_f(\bm{Q}^{n+1}_{i,j}) \approx \bm{Q}^{n+1}_{i,j} \tilde{\bm{S}_f}(\bm{Q}^n_{i,j}).$$

When we then solve the semi-discretized ODE through a second-order TVD Runge-Kutta method, we gather the updated terms on the left-hand side, so that we get:
$$\begin{split}
Q^{*}_{i,j} &= \left[ Q^n_{i,j}  + \Delta t R(Q^n)_{i,j} \right] / \left[ 1 - \Delta t \tilde{S_f}(Q^{n}_{i,j}) \right] \\
Q^{n+1}_{i,j} &= \Big[\tfrac{1}{2}Q^{n}_{i,j} + \tfrac{1}{2}\left[ Q^{*}_{i,j}  - \Delta t R(Q^{*})_{i,j} \right] \Big]/ \left[ 1 + \tfrac{1}{2}\Delta t \tilde{S_f}(Q^{*}_{i,j}) \right] \\
\end{split}
$$

Keep in mind the CFL conditions
$$\Delta t \leq \tfrac{1}{4}\min\left\{\frac{\Delta x}{\max_{\Omega}\left( |u| + \sqrt{gh}\right) },\; \frac{\Delta y}{\max_{\Omega}\left( |v| + \sqrt{gh}\right) }  \right\}
$$



### Open questions and TODOs:
* [ ] Automatic timesteps
* [ ] Should friction depend on grid size?


### References
* Kurganov, Petrova (2007) *A second-order well-balanced positivity preserving central-upwind scheme for the Saint-Venant system*. Communications in Mathematical Sciences, 5:133-160. [link](https://projecteuclid.org/journals/communications-in-mathematical-sciences/volume-5/issue-1/A-Second-Order-Well-Balanced-Positivity-Preserving-Central-Upwind-Scheme/cms/1175797625.full)
* Brodtkorb, Holm (2021) *Coastal ocean forecasting on the GPU using a two-dimensional finite-volume scheme*. Tellus A: Dynamic Meteorology and Oceanography, 73:1, 1-22, DOI: [10.1080/16000870.2021.1876341](https://doi.org/10.1080/16000870.2021.1876341)
* Brodtkorb, Sætra, Altinakar,(2012) *Efficient shallow water simulations on GPUs: Implementation, visualization, verification, and validation*. Comput. Fluids 55, 1–12. DOI: [10.1016/j.compfluid.2011.10.012](https://doi.org/10.1016/j.compfluid.2011.10.012)
* Fernandez-Pato, Caviedes-Voullieme, Garcia-Navarro (2016) *Rainfall/runoff simulation with 2D full shallow water equations: Sensitivity analysis and calibration of infiltration parameters*. Journal of Hydrology
536, 496-513 DOI: [10.1016/j.jhydrol.2016.03.021](https://doi.org/10.1016/j.jhydrol.2016.03.021)
* Brodtkorb, Hagen, Lie, Natvig (2010) *Simulation and visualization of the Saint-Venant system using GPUs*. Comput. Visual Sci. 13, 341–353 DOI: [10.1007/s00791-010-0149-x](https://doi.org/10.1007/s00791-010-0149-x)